In [13]:
import tweepy
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import requests as reqs

In [14]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [15]:
consumer_key = "tIZ2XiI8vNVadgJbcraPofPDU"
consumer_secret = "illoEp1AY8WCnmLrCwD1yUrrCgB9w5XUFStgXVShGhlphiWeeC"
access_token = "937017463102832640-8nTjGBdVxZA1OnhLEgw8js5hYWWo58o"
access_token_secret = "HU4lM9ZTeDxi5Qp9rtm4WrMQXABrANSB6OXt3NlLwDCPw"

In [16]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [17]:
news_orgs = ["@bbcnews", "@CBSNews", "@foxnews", "@cnnbrk", "@nytimes"]

In [18]:
sentiments = []

In [19]:
compound_list = []
positive_list = []
negative_list = []
neutral_list = []

In [39]:
for news in news_orgs:

    counter = 1
    
    for x in range(5):
    
        public_tweets = api.user_timeline(news, page = x)
    
        for tweet in public_tweets:
        
            compound = analyzer.polarity_scores(tweet["text"])["compound"]
            pos = analyzer.polarity_scores(tweet["text"])["pos"]
            neu = analyzer.polarity_scores(tweet["text"])["neu"]
            neg = analyzer.polarity_scores(tweet["text"])["neg"]
            tweets_ago = counter
            text = tweet['text']
        
            compound_list.append(compound)
            positive_list.append(pos)
            negative_list.append(neg)
            neutral_list.append(neu)
        
    sentiments.append({"Source": news,
                           "Date": tweet["created_at"], 
                           "Text": text,
                           "Compound": compound,
                           "Positive": pos,
                           "Negative": neu,
                           "Neutral": neg,
                           "Tweets Ago": counter})
        
    counter = counter + 1



AttributeError: 'JSONParser' object has no attribute 'model_factory'

In [12]:
for news in news_orgs:
    counter = 1      
    for item in tweepy.Cursor(api.user_timeline, id=news).items(100):
        tweet = json.dumps(item._json, indent=3)
        tweet = json.loads(tweet)
        text = tweet['text']
        
        compound = analyzer.polarity_scores(tweet["text"])["compound"]
        pos = analyzer.polarity_scores(tweet["text"])["pos"]
        neu = analyzer.polarity_scores(tweet["text"])["neu"]
        neg = analyzer.polarity_scores(tweet["text"])["neg"]
        
        news_dict = {
            'Source':news,
            'Date': tweet["created_at"],
            'Text': text,
            'Compound': compound,
            'Positive': pos,
            'Negative': neg,
            'Tweets_Ago': counter
        }
        
        sentiments.append(news_dict)
        counter +=1      


AttributeError: 'JSONParser' object has no attribute 'model_factory'

In [34]:
#create dataframe for sentiment dictionary
news_sentiment_data = pd.DataFrame.from_dict(sentiments)

#Display data from news_sentiment dataframe
news_sentiment_data.head()

,Compound,Date,Negative,Neutral,Positive,Source,Text,Tweets Ago
0,-0.5574,Wed Dec 13 07:59:18 +0000 2017,0.874,0.126,0.000,@bbcnews,RT @BBCBreakfast: Hundreds of Star Wars fans l...,1
1,0.4865,Wed Dec 13 14:30:31 +0000 2017,0.864,0.000,0.136,@CBSNews,"""I've never met any president that has more ch...",1
2,0.7906,Wed Dec 13 20:29:39 +0000 2017,0.692,0.000,0.308,@foxnews,"Pastor Leon Benjamin: ""It's time that we recog...",1
3,-0.3612,Sat Dec 09 03:40:02 +0000 2017,0.889,0.111,0.000,@cnnbrk,RT @CNNPolitics: President Trump responds to c...,1
4,-0.0191,Wed Dec 13 18:09:05 +0000 2017,0.944,0.056,0.000,@nytimes,But enough about the humans. What about the po...,1


In [33]:
sentiments_df.groupby('Source').Source.count()

Source
@CBSNews    3
@bbcnews    4
@cnnbrk     3
@foxnews    3
@nytimes    3
Name: Source, dtype: int64

In [ ]:
BBC_graph_data = news_sentiment_data[news_sentiment_data["Source"]=="@BBC"]
CBS_graph_data = news_sentiment_data[news_sentiment_data["Source"]=="@CBS"]
CNN_graph_data = news_sentiment_data[news_sentiment_data["Source"]=="@CNN"]
FoxNews_graph_data = news_sentiment_data[news_sentiment_data["Source"]=="@FoxNews"]
nytimes_graph_data = news_sentiment_data[news_sentiment_data["Source"]=="@nytimes"]

In [ ]:
BBC_graph = sns.lmplot(x = 'Tweets Ago', y = 'Compound', data = BBC_graph_data, fit_reg = False, hue =  'User')
CBS_graph = sns.lmplot(x = 'Tweets Ago', y = 'Compound', data = CBS_graph_data, fit_reg = False, hue =  'User')
CNN_graph = sns.lmplot(x = 'Tweets Ago', y = 'Compound', data = CNN_graph_data, fit_reg = False, hue =  'User')
FoxNews_graph = sns.lmplot(x = 'Tweets Ago', y = 'Compound', data = FoxNews_graph_data, fit_reg = False, hue =  'User')
nytimes_graph = sns.lmplot(x = 'Tweets Ago', y = 'Compound', data = nytimes_graph, fit_reg = False, hue =  'User')

plt.show()

In [ ]:
# New graph df for news sentiment compound score

News_graph = news_sentiment_data.groupby(['Source'])['Compound'].mean()
News_graph = pd.DataFrame(News_graph)
News_graph

In [ ]:
News = ['BBC', 'CBS', 'CNN', 'FoxNews', 'New York Times']
colors = ['lightskyblue', 'green', 'red', 'blue', 'yellow']
Tweet_Polarity = News_graph['Compound']
ypos = np.arange(len(News))
plt.xticks(ypos,News)

plt.bar(y_pos, Tweet_Polarity, align='center', alpha=0.5, color = colors)
 
# Label x and y axis
plt.xlabel("News Source")
plt.ylabel("Tweet_Polarity (Compound Sentiments Score)")

# Prints our bar graph to the screen
plt.title("Overall Media Sentiment based on Twitter - (time.strftime("%d/%m/%Y"))")
plt.show()

In [ ]:
plt.plot(np.arange(len(sentiments_df["Compound"])),
         sentiments_df["Compound"], marker="o", linewidth=0.5, linestyle = 'None',
         alpha=0.8)

# # Incorporate the other graph properties
plt.title("Sentiment Analysis of Tweets (%s) for %s" % (time.strftime("%x"), news_org))
plt.ylabel("Tweet Polarity")
plt.xlabel("Tweets Ago")
plt.show()